# Predictive Maintenance using Deep Learning
40 relevant attributes out of 85 were identified to be used as input to a recurrent neural network. We trained a baseline model that would “detect” anomalous events on each node as soon as they occur with a 10-fold cross validated AUC of 0.97. 

| #  | Traffic load | No. Anomalies | Duration | Description                        |
|----|--------------|---------------|----------|------------------------------------|
| 0  | 0            | 0             | 1h       | Baseline (no amolies)              |
| 1  | 500Gbps      | 0             | 1h       | Baseline (no anomalies)            |
| 2  | 1Tbps        | 11            | 1h       | BGP Clear                          |
| 3  | 1Tbps        | 8             | 0.55h    | BGP Clear                          |
| 4  | 1Tbps        | 5             | 0.72h    | Port Flap                          |
| 5  | 1Tbps        | 12            | 2h       | BGP Clear                          |
| 6  | 0            | 12            | 2h       | BGP Clear                          |
| 7  | 0            | 130           | 72h      | (VIRL) BGP Clear                   |
| 8  | 0            | 238           | 262h     | (VIRL) BGP Clear                   |
| 9  | 2.9Tbps      | 5             | .75h     | Port Admin Shut                    |
| 10 | 2Tbps        | 5             | .55h     | Port Transceiver Pull and Reinsert |

Data source: [Cisco real-world telemetry anomaly dataset](https://github.com/cisco-ie/telemetry) which includes timeseries of 85 signals from 12 nodes under 0 to 1Tbps traffic of Blue Ray/Youtube4K movie streaming, skype video calls and other activities. BGP Clear events from dataset 2, 3, 5, 6 are used for modeling for this work.

The preprocessed data is available in [here in s3](https://s3.amazonaws.com/cisco-anomaly-detection/preprocessed_20190401.pkl) by this [notebook](./cisco_telemetry_data_preprocessing_20190401.ipynb). Implementation and training was performed on an AWS EC2 p2.xlarge instance.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import sys, os, warnings, pickle
warnings.filterwarnings("ignore")
%matplotlib inline

import sklearn.model_selection as skms
import sklearn.metrics as skm
import sklearn.preprocessing as skp

import tensorflow as tf
# Here we would like to restrict the GPU memory usage by tensorflow which by default 
# would allocate all memory available to this routine. We ask it to allocate what is 
# necessary during runtime and allow it to grow. Note that this configuration has to
# be set prior to importing keras to have the intended effect. 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

import keras.backend as K
from keras.layers import Dense, Dropout, LSTM, TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping, Callback
from keras.models import Sequential

exp='nn_keras_cv_20190401'
logdir='/home/ubuntu/Projects/cisco/logs/%s' % exp

try:
    os.makedirs(logdir)
except:
    pass

np.random.seed(1)
tf.set_random_seed(2)

Using TensorFlow backend.


Getting the preprocessed data from pickle

In [51]:
d_df_resample=pickle.load(open('../Data/cisco_telemetry/preprocessed_20190401.pkl', 'rb'))

In [52]:
d_df_resample['2_leaf1_0'].columns

Index(['active-routes-count', 'as', 'backup-routes-count', 'bandwidth',
       'bytes-received', 'bytes-sent', 'carrier-transitions', 'crc-errors',
       'deleted-routes-count', 'free-application-memory',
       'free-physical-memory', 'global__established-neighbors-count-total',
       'global__neighbors-count-total', 'global__nexthop-count',
       'input-data-rate', 'input-drops', 'input-errors', 'input-load',
       'input-packet-rate', 'load-interval', 'no-route-packets',
       'output-data-rate', 'output-drops', 'output-load', 'output-packet-rate',
       'packets-received', 'packets-sent', 'paths-count',
       'performance-statistics__global__configuration-items-processed',
       'performance-statistics__vrf__inbound-update-messages',
       'protocol-route-memory', 'reliability', 'routes-counts',
       'total-cpu-fifteen-minute', 'total-cpu-five-minute',
       'total-cpu-one-minute', 'vrf__neighbors-count', 'vrf__network-count',
       'vrf__path-count', 'vrf__update-mess

Extract the feature columns and target columns. There are shifted anomaly labels in the dataset for other purposes in the future. For now we are going to model labels based on actual anomaly, `Anomaly`.

In [55]:
feature_cols=d_df_resample['2_leaf1_0'].columns[:-11]
target_cols=[i for i in d_df_resample['2_leaf1_0'].columns if 'Anomaly' in i]

In [56]:
feature_cols

Index(['active-routes-count', 'as', 'backup-routes-count', 'bandwidth',
       'bytes-received', 'bytes-sent', 'carrier-transitions', 'crc-errors',
       'deleted-routes-count', 'free-application-memory',
       'free-physical-memory', 'global__established-neighbors-count-total',
       'global__neighbors-count-total', 'global__nexthop-count',
       'input-data-rate', 'input-drops', 'input-errors', 'input-load',
       'input-packet-rate', 'load-interval', 'no-route-packets',
       'output-data-rate', 'output-drops', 'output-load', 'output-packet-rate',
       'packets-received', 'packets-sent', 'paths-count',
       'performance-statistics__global__configuration-items-processed',
       'performance-statistics__vrf__inbound-update-messages',
       'protocol-route-memory', 'reliability', 'routes-counts',
       'total-cpu-fifteen-minute', 'total-cpu-five-minute',
       'total-cpu-one-minute', 'vrf__neighbors-count', 'vrf__network-count',
       'vrf__path-count', 'vrf__update-mess

In [58]:
n_timesteps = d_df_resample['2_leaf1_0'].shape[0]
n_features = len(feature_cols)

Profiling the data:

In [59]:
n_timesteps, n_features

(85, 40)

In [30]:
d_df_resample['2_leaf1_0']['Anomaly'].value_counts()

False    67
True     18
Name: Anomaly, dtype: int64

In [31]:
d_df_resample['2_leaf1_0']['Anomaly'].head()

time
2017-06-29 16:34:30    False
2017-06-29 16:34:40    False
2017-06-29 16:34:50    False
2017-06-29 16:35:00    False
2017-06-29 16:35:10    False
Freq: 10S, Name: Anomaly, dtype: bool

Filter samples by having anomaly or not into two dictionary. We will be using positive samples (with anomalies) for training and evaluation.

In [60]:
d_df_resample_positive={}
d_df_resample_negative={}
for key, df in d_df_resample.items():
    #print(key)
    if df['Anomaly'].any():
        d_df_resample_positive[key]=df
    else:
        d_df_resample_negative[key]=df

Creating input dataset, `X` and `Y`, and `group` for storing dataset information which will later be used to stratify the data during the cross-validation.

In [65]:
X=np.zeros((len(d_df_resample_positive.keys()), n_timesteps, n_features))
Y=np.zeros((len(d_df_resample_positive.keys()), n_timesteps))
Y_orig=np.zeros_like(Y)
target='Anomaly'
group=[]

for i, node in enumerate(d_df_resample_positive.keys()):
    X[i] = d_df_resample_positive[node][feature_cols].values
    Y[i] = d_df_resample_positive[node][target].values
    Y_orig[i] = d_df_resample_positive[node]['Anomaly_orig'].values
    group.append(node.split('_')[0])
group=np.array(group)

In [68]:
X.shape, Y.shape, Y_orig.shape, i, node

((48, 85, 40), (48, 85), (48, 85), '0_0', '6_leaf3_8')

In [70]:
Y_orig[0], Y[0]

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [41]:
group

array(['2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '3',
       '3', '3', '3', '3', '3', '3', '3', '5', '5', '5', '5', '5', '5',
       '5', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6',
       '6', '6', '6', '6', '6', '6', '6', '6', '6'], dtype='<U1')

In [87]:
def visualize_prediction_clustermap(df, columns=None, **kw):
    row_colors=df[[target, 'Pred']].replace(to_replace={True: 'r', False: 'b'})
    g=sns.clustermap(df[columns].T, col_colors=row_colors, figsize=(16,10), vmin=-8, vmax=8, **kw)
    return g

## Training and Evaluation using 10-fold cross-validation 

In [44]:
# define the stratified 10-fold iterator
skfold = skms.StratifiedKFold(n_splits=10, random_state=0)

# initialize the scoring metrics 
# acc: accuracy, auc: area under roc curve, mse: mean squared error, 
# ba: balanced precision, recall, precison and f1.
acc_cv_train=np.zeros(10)
acc_cv_test=np.zeros_like(acc_cv_train)
d_acc_cv={'train': acc_cv_train, 'test': acc_cv_test}

auc_cv_train=np.zeros_like(acc_cv_train)
auc_cv_test=np.zeros_like(acc_cv_train)
d_auc_cv={'train': auc_cv_train, 'test': auc_cv_test}

mse_cv_train=np.zeros_like(acc_cv_train)
mse_cv_test=np.zeros_like(acc_cv_train)
d_mse_cv={'train': mse_cv_train, 'test': mse_cv_test}

recall_cv_train=np.zeros_like(acc_cv_train)
recall_cv_test=np.zeros_like(recall_cv_train)
d_recall_cv={'train': recall_cv_train, 'test': recall_cv_test}

precision_cv_train=np.zeros_like(acc_cv_train)
precision_cv_test=np.zeros_like(precision_cv_train)
d_precision_cv={'train': precision_cv_train, 'test': precision_cv_test}

f1_cv_train=np.zeros_like(acc_cv_train)
f1_cv_test=np.zeros_like(f1_cv_train)
d_f1_cv={'train': f1_cv_train, 'test': f1_cv_test}

ba_cv_train=np.zeros_like(acc_cv_train)
ba_cv_test=np.zeros_like(ba_cv_train)
d_ba_cv={'train': ba_cv_train, 'test': ba_cv_test}

# dictionaries to hold intermediate data and output data
d_xtrain={}
d_xtest={}
d_models = {}

# define early stopping criteria for training to avoid overfitting
earlystopping = EarlyStopping(monitor='val_mean_squared_error', patience=10, min_delta=0.0001, verbose=1, mode='auto')

# stratified 10-fold X-val, within each fold, a new model will be created and training using 9 folds of data
for i, (train_index, test_index) in enumerate(skfold.split(X, group)):
    print('%d fold' % (i+1))
    x_train = X[train_index]
    x_test = X[test_index]
    
    # because of the nature of time-series prediction, the target needs to be of shape (n_samples, n_timesteps, 1)
    y_train = Y[train_index].reshape(-1, n_timesteps, 1)  
    y_test = Y[test_index].reshape(-1, n_timesteps, 1)
    
    # preprocessing the data: z-score standardization across time for each feature
    scaler = skp.StandardScaler()
    x_train = scaler.fit_transform(x_train.reshape((len(train_index)*n_timesteps, n_features))) \
                                    .reshape((len(train_index), n_timesteps, n_features))
    x_test = scaler.transform(x_test.reshape((len(test_index)*n_timesteps, n_features))) \
                                    .reshape((len(test_index), n_timesteps, n_features))
    d_xtrain[i]=x_train
    d_xtest[i]=x_test
    
    d_x = {'train': x_train, 'test': x_test}
    d_y = {'train': y_train, 'test': y_test}
    
    # building the model:
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps, n_features)))
    model.add(Dropout(0.2))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    
    # only print summary during the first fold
    if i == 0:
        print(model.summary())
    
    # logging into tensorboard
    cv_dir = 'cv%d' % i
    cv_dir = os.path.join(logdir, cv_dir)
    tensorboard = TensorBoard(log_dir=cv_dir)
    
    # optimize using Adam with loss for binary prediction
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0007), metrics=['mean_squared_error', 'accuracy'])  
    
    # training. use 8% of training samples as validation set.
    model.fit(x_train, y_train, shuffle=True, validation_split = 0.08,
               batch_size = 1, epochs=150, verbose=2, callbacks=[earlystopping, tensorboard]) #, , aucroc])

    d_models[i]=model
    
    print('')
    print('AUC, ACC, MSE, RECALL, PRECISION, F1, BALANCED_ACC:')
    for key in d_acc_cv.keys():
        pred=model.predict(d_x[key])
        d_auc_cv[key][i] = skm.roc_auc_score(d_y[key].flatten(), pred.flatten())
        d_acc_cv[key][i] = skm.accuracy_score(d_y[key].flatten(), pred.flatten()>0.5)
        d_mse_cv[key][i] = skm.mean_squared_error(d_y[key].flatten(), pred.flatten())
        d_recall_cv[key][i] = skm.recall_score(d_y[key].flatten(), pred.flatten()>0.5)
        d_precision_cv[key][i] = skm.precision_score(d_y[key].flatten(), pred.flatten()>0.5)
        d_f1_cv[key][i] = skm.f1_score(d_y[key].flatten(), pred.flatten()>0.5)
        d_ba_cv[key][i] = skm.balanced_accuracy_score(d_y[key].flatten(), pred.flatten()>0.5)
        print('\t%s: %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f' % \
              (key, d_auc_cv[key][i], d_acc_cv[key][i], d_mse_cv[key][i], d_recall_cv[key][i], 
                    d_precision_cv[key][i], d_f1_cv[key][i], d_ba_cv[key][i]))

1 fold
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 85, 100)           56400     
_________________________________________________________________
dropout_15 (Dropout)         (None, 85, 100)           0         
_________________________________________________________________
dense_15 (Dense)             (None, 85, 80)            8080      
_________________________________________________________________
dropout_16 (Dropout)         (None, 85, 80)            0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 85, 1)             81        
Total params: 64,561
Trainable params: 64,561
Non-trainable params: 0
_________________________________________________________________
None
Train on 37 samples, validate on 4 samples
Epoch 1/150
 - 6s - loss: 0.6155 - mean_squared_error: 0.2134 - acc: 0.6343 - val_l

Epoch 11/150
 - 5s - loss: 0.0896 - mean_squared_error: 0.0264 - acc: 0.9659 - val_loss: 0.4049 - val_mean_squared_error: 0.1376 - val_acc: 0.8088
Epoch 12/150
 - 5s - loss: 0.0857 - mean_squared_error: 0.0256 - acc: 0.9647 - val_loss: 0.5247 - val_mean_squared_error: 0.1602 - val_acc: 0.8118
Epoch 13/150
 - 5s - loss: 0.0678 - mean_squared_error: 0.0199 - acc: 0.9746 - val_loss: 0.6467 - val_mean_squared_error: 0.1721 - val_acc: 0.8088
Epoch 00013: early stopping

AUC, ACC, MSE, RECALL, PRECISION, F1, BALANCED_ACC:
	train: 0.9910, 0.9608, 0.0321, 0.9045, 0.8858, 0.8951, 0.9390
	test: 0.9843, 0.9529, 0.0405, 0.8611, 0.8158, 0.8378, 0.9146
4 fold
Train on 39 samples, validate on 4 samples
Epoch 1/150
 - 7s - loss: 0.5779 - mean_squared_error: 0.1969 - acc: 0.6887 - val_loss: 0.4050 - val_mean_squared_error: 0.1165 - val_acc: 0.8824
Epoch 2/150
 - 5s - loss: 0.3224 - mean_squared_error: 0.0947 - acc: 0.8706 - val_loss: 0.2671 - val_mean_squared_error: 0.0678 - val_acc: 0.9618
Epoch 3/150

Epoch 18/150
 - 5s - loss: 0.0612 - mean_squared_error: 0.0179 - acc: 0.9762 - val_loss: 0.0868 - val_mean_squared_error: 0.0230 - val_acc: 0.9765
Epoch 19/150
 - 5s - loss: 0.0551 - mean_squared_error: 0.0163 - acc: 0.9777 - val_loss: 0.0940 - val_mean_squared_error: 0.0250 - val_acc: 0.9735
Epoch 20/150
 - 5s - loss: 0.0896 - mean_squared_error: 0.0249 - acc: 0.9716 - val_loss: 0.1026 - val_mean_squared_error: 0.0260 - val_acc: 0.9735
Epoch 21/150
 - 5s - loss: 0.0748 - mean_squared_error: 0.0218 - acc: 0.9710 - val_loss: 0.0402 - val_mean_squared_error: 0.0104 - val_acc: 0.9853
Epoch 22/150
 - 5s - loss: 0.0679 - mean_squared_error: 0.0198 - acc: 0.9741 - val_loss: 0.2715 - val_mean_squared_error: 0.0898 - val_acc: 0.8500
Epoch 23/150
 - 5s - loss: 0.0756 - mean_squared_error: 0.0222 - acc: 0.9698 - val_loss: 0.1242 - val_mean_squared_error: 0.0385 - val_acc: 0.9588
Epoch 24/150
 - 5s - loss: 0.0691 - mean_squared_error: 0.0200 - acc: 0.9738 - val_loss: 0.1267 - val_mean_squared_err

Epoch 12/150
 - 24s - loss: 0.0772 - mean_squared_error: 0.0226 - acc: 0.9703 - val_loss: 0.4031 - val_mean_squared_error: 0.1227 - val_acc: 0.8529
Epoch 13/150
 - 25s - loss: 0.0770 - mean_squared_error: 0.0228 - acc: 0.9691 - val_loss: 0.2621 - val_mean_squared_error: 0.0915 - val_acc: 0.8735
Epoch 14/150
 - 28s - loss: 0.0747 - mean_squared_error: 0.0225 - acc: 0.9668 - val_loss: 0.2158 - val_mean_squared_error: 0.0767 - val_acc: 0.8853
Epoch 15/150
 - 28s - loss: 0.0614 - mean_squared_error: 0.0182 - acc: 0.9735 - val_loss: 0.2493 - val_mean_squared_error: 0.0871 - val_acc: 0.8765
Epoch 16/150
 - 28s - loss: 0.0568 - mean_squared_error: 0.0173 - acc: 0.9753 - val_loss: 0.3450 - val_mean_squared_error: 0.1091 - val_acc: 0.8647
Epoch 17/150
 - 28s - loss: 0.0600 - mean_squared_error: 0.0183 - acc: 0.9724 - val_loss: 0.2653 - val_mean_squared_error: 0.0898 - val_acc: 0.8735
Epoch 00017: early stopping

AUC, ACC, MSE, RECALL, PRECISION, F1, BALANCED_ACC:
	train: 0.9947, 0.9676, 0.0237,

 - 28s - loss: 0.0575 - mean_squared_error: 0.0171 - acc: 0.9776 - val_loss: 0.0228 - val_mean_squared_error: 0.0068 - val_acc: 0.9882
Epoch 18/150
 - 28s - loss: 0.0542 - mean_squared_error: 0.0169 - acc: 0.9759 - val_loss: 0.0189 - val_mean_squared_error: 0.0051 - val_acc: 0.9941
Epoch 19/150
 - 29s - loss: 0.0571 - mean_squared_error: 0.0164 - acc: 0.9770 - val_loss: 0.0195 - val_mean_squared_error: 0.0057 - val_acc: 0.9941
Epoch 20/150
 - 29s - loss: 0.0609 - mean_squared_error: 0.0186 - acc: 0.9719 - val_loss: 0.0461 - val_mean_squared_error: 0.0145 - val_acc: 0.9794
Epoch 21/150
 - 28s - loss: 0.0529 - mean_squared_error: 0.0147 - acc: 0.9819 - val_loss: 0.0370 - val_mean_squared_error: 0.0120 - val_acc: 0.9824
Epoch 22/150
 - 29s - loss: 0.0501 - mean_squared_error: 0.0145 - acc: 0.9813 - val_loss: 0.0181 - val_mean_squared_error: 0.0052 - val_acc: 0.9912
Epoch 23/150
 - 27s - loss: 0.0453 - mean_squared_error: 0.0128 - acc: 0.9842 - val_loss: 0.0187 - val_mean_squared_error: 0.

Average performance over 10 folds. Metrics shown are AUC, Accuracy, MSE, Recall, Precision, F1, and Balanced Accuracy, respectively.

In [45]:
for score in [d_auc_cv, d_acc_cv, d_mse_cv, d_recall_cv, d_precision_cv, d_f1_cv, d_ba_cv]:
    for j in ['train', 'test']:
        print('%s: %.3f (%.3f)' % (j, score[j].mean(), score[j].std()))
    print()

train: 0.995 (0.007)
test: 0.972 (0.032)

train: 0.972 (0.021)
test: 0.916 (0.078)

train: 0.022 (0.018)
test: 0.066 (0.064)

train: 0.938 (0.027)
test: 0.835 (0.145)

train: 0.913 (0.075)
test: 0.792 (0.201)

train: 0.924 (0.052)
test: 0.795 (0.157)

train: 0.959 (0.023)
test: 0.884 (0.079)



Visualize the data and cross-validated prediction.

In [ ]:
list_keys=list(d_df_resample_norm_cv.keys())
sns.set_context('talk')
for i in list_keys:
#     print(i)
    visualize_prediction_clustermap(d_df_resample_norm_cv[i], columns=feature_cols, 
                                    row_cluster=True, col_cluster=False, method='centroid')

This will create figures where horizontal axis is time and vertical axis is features which are clustered to show similarity and correlation to the anomaly. Ground truth labels is shown up top as `Anomaly` while `Pred` is our recurrent network prediction.
![prediction](https://s3.amazonaws.com/cisco-anomaly-detection/prediction_clustermap_example_1.png)

--------
## Appendix

In [46]:
cv_test_key=[i for _, i in skfold.split(X, group)]
cv_train_key=[i for i, _ in skfold.split(X, group)]

In [47]:
cv_test_key

[array([ 0,  1, 12, 20, 21, 35, 36]),
 array([ 2,  3, 13, 22, 23, 37, 38]),
 array([ 4, 14, 24, 25, 39, 40]),
 array([ 5, 15, 26, 27, 41]),
 array([ 6, 16, 28, 29, 42]),
 array([ 7, 17, 30, 43]),
 array([ 8, 18, 31, 44]),
 array([ 9, 19, 32, 45]),
 array([10, 33, 46]),
 array([11, 34, 47])]

In [48]:
cv_train_key

[array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19,
        22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 39, 40,
        41, 42, 43, 44, 45, 46, 47]),
 array([ 0,  1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18, 19,
        20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 39, 40,
        41, 42, 43, 44, 45, 46, 47]),
 array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 41, 42, 43, 44, 45, 46, 47]),
 array([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 39, 40, 42, 43, 44, 45, 46, 47]),
 array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 39, 40, 41, 43, 44, 45, 46, 47]),
 array([ 0,  1,  2,  3,  4,  5

In [66]:
d_df_resample_norm_cv={}
for i in range(10):
    cv_pred=d_models[i].predict(d_xtest[i])
    for j in range(cv_pred.shape[0]):
        key = '%s_%s' % (i, j)
        test_ind = cv_test_key[i][j]
        d_df_resample_norm_cv[key] = d_df_resample['2_leaf1_0'].copy()
        d_df_resample_norm_cv[key].index = d_df_resample_norm_cv[key]['time']
        d_df_resample_norm_cv[key][feature_cols] = d_xtest[i][j]
        d_df_resample_norm_cv[key]['Anomaly'] = Y[test_ind].astype(bool)
        d_df_resample_norm_cv[key]['Anomaly_orig'] = Y_orig[test_ind].astype(bool)
        d_df_resample_norm_cv[key]['Pred'] = cv_pred[j].flatten()>0.5

In [85]:
d_df_resample_norm_cv[i].index

DatetimeIndex(['2017-06-29 16:34:30', '2017-06-29 16:34:40',
               '2017-06-29 16:34:50', '2017-06-29 16:35:00',
               '2017-06-29 16:35:10', '2017-06-29 16:35:20',
               '2017-06-29 16:35:30', '2017-06-29 16:35:40',
               '2017-06-29 16:35:50', '2017-06-29 16:36:00',
               '2017-06-29 16:36:10', '2017-06-29 16:36:20',
               '2017-06-29 16:36:30', '2017-06-29 16:36:40',
               '2017-06-29 16:36:50', '2017-06-29 16:37:00',
               '2017-06-29 16:37:10', '2017-06-29 16:37:20',
               '2017-06-29 16:37:30', '2017-06-29 16:37:40',
               '2017-06-29 16:37:50', '2017-06-29 16:38:00',
               '2017-06-29 16:38:10', '2017-06-29 16:38:20',
               '2017-06-29 16:38:30', '2017-06-29 16:38:40',
               '2017-06-29 16:38:50', '2017-06-29 16:39:00',
               '2017-06-29 16:39:10', '2017-06-29 16:39:20',
               '2017-06-29 16:39:30', '2017-06-29 16:39:40',
               '2017-06-